In [ ]:
#manage imports (will adjust as needed)
from llama_index.core import (
    VectorStoreIndex,
    SummaryIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    Settings,
)
from dotenv import load_dotenv
from openai import OpenAI
from llama_index.core.schema import IndexNode
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
load_dotenv()
# Local llm for query engine
api_key = 'ollama'
model = "llama3.2:latest"
base_url = 'http://localhost:11434/v1/'

client = OpenAI(
    base_url=base_url,
    api_key=api_key
)
Settings.llm = client
Settings.embed_model = embed_model
Settings.chunk_size = 512

Document text extraction and processing


In [ ]:
import os

def append_supported_filenames(directory_path, file_list, extensions=(".txt", ".pdf", ".epub")):
    """
    Appends the names of supported files (e.g., .txt, .pdf, .epub) in the given directory to the provided list.

    Args:
        directory_path (str): The path to the directory containing the files.
        file_list (list): The list to which file names will be appended.
        extensions (tuple): A tuple of allowed file extensions. Default is (".txt", ".pdf", ".epub").

    Returns:
        None: Modifies the input list in place.
    """
    if not os.path.isdir(directory_path):
        raise ValueError(f"The path '{directory_path}' is not a valid directory.")
    
    for item in os.listdir(directory_path):
        if os.path.isfile(os.path.join(directory_path, item)) and item.lower().endswith(extensions):
            file_list.append(item)

In [ ]:
documents = [] #suppose this is a list of the name of the documents in the directory "documents"
append_supported_filenames("/documents", documents)
extracted_docs = {}
for doc in documents:
    extracted_docs[doc] = SimpleDirectoryReader(
        input_files = [f"documents/{doc}"],
    ).load_data()